In [ ]:
import cv2
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO #for input/output stream file related
import random
import json
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

model=load_model('facerecognition_model.h5')

face_detect=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#load function
def face_extractor(img):
    
    #this function is for detect faces and it will return cropped images of faces 
    
    #if no face detected it will return input image
    
    
    faces=face_detect.detectMultiScale(img,1.3,5)
    
    if faces == ():
        return None
    
    #below function for returning the face
    for (x,y,w,h) in faces:
        x=x-10
        y=y-10
        cropped_face=img[y:y+h+50,x:x+w+50]
        return cropped_face
    


In [ ]:
cap=cv2.VideoCapture(0)

while True:
    _,frame=cap.read()
    
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face=cv2.resize(face,(224,224))
        #resize the image because our image takes the input of that
        #this line will convert face array in image rgb
        im=Image.fromarray(face,'RGB')
        
        img_array=np.array(im)
        
        #our keras uses 4d tensor , example(no. of images* height * width * channel)
        
        #we have to change image dimensions from 1,224,224 to (1,224,224,3)
        
        img_array=np.expand_dims(img_array,axis=0)
        
        
        #make the predictions
        pred=model.predict(img_array)
        
        print(pred)
        
        Note='Not matching'
        if(pred[0][1]>0.5):
            Note='1st class'
        if(pred[0][2]>0.5):
            Note='2nd class'

        cv2.putText(frame,Note,(20,20),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,255),2)
        
    else:
        cv2.putText(frame,"no face found sorry",(20,20),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,255),2)
        
    cv2.imshow('Face Recognition',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
            